In [1]:
!git clone https://github.com/AI4Bharat/IndicTrans2.git

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 668 (delta 108), reused 77 (delta 70), pack-reused 517
Receiving objects: 100% (668/668), 4.12 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (415/415), done.


In [2]:
%cd /content/IndicTrans2/huggingface_interface

/content/IndicTrans2/huggingface_interface


In [3]:
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 45.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor,IndicTransTokenizer

In [5]:
BATCH_SIZE=4
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
quantization=None

In [6]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [8]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    "Hello , My name is Shikhar Dave",
    "I am studying neural machine Translation",
    "I hail from jodhpur , rajasthan",
    "I am a student at IIT Jodhpur",

]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


eng_Latn - hin_Deva
eng_Latn: Hello , My name is Shikhar Dave
hin_Deva: हैलो, मेरा नाम शिखर डेव है।
eng_Latn: I am studying neural machine Translation
hin_Deva: मैं तंत्रिका यंत्र अनुवाद का अध्ययन कर रहा हूँ।
eng_Latn: I hail from jodhpur , rajasthan
hin_Deva: मैं जोधपुर, राजस्थान का रहने वाला हूँ।
eng_Latn: I am a student at IIT Jodhpur
hin_Deva: मैं आई. आई. टी. जोधपुर का छात्र हूँ।


In [10]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)


en_sents = [
    "Hello , My name is Shikhar Dave",
    "I am studying neural machine Translation",
    "I hail from jodhpur , rajasthan",
    "I am a student at IIT Jodhpur",

]

src_lang, tgt_lang = "eng_Latn", "guj_Gujr"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


eng_Latn - guj_Gujr
eng_Latn: Hello , My name is Shikhar Dave
guj_Gujr: નમસ્તે, મારું નામ શિખર ડેવ છે.
eng_Latn: I am studying neural machine Translation
guj_Gujr: હું ન્યુરલ મશીન ટ્રાન્સલેશનનો અભ્યાસ કરી રહ્યો છું.
eng_Latn: I hail from jodhpur , rajasthan
guj_Gujr: હું જોધપુર, રાજસ્થાનનો રહેવાસી છું.
eng_Latn: I am a student at IIT Jodhpur
guj_Gujr: હું આઈ. આઈ. ટી. જોધપુરનો વિદ્યાર્થી છું.


In [11]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)


en_sents = [
    "Hello , My name is Shikhar Dave",
    "I am studying neural machine Translation",
    "I hail from jodhpur , rajasthan",
    "I am a student at IIT Jodhpur",

]

src_lang, tgt_lang = "eng_Latn", "ben_Beng"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


eng_Latn - ben_Beng
eng_Latn: Hello , My name is Shikhar Dave
ben_Beng: হ্যালো, আমার নাম শিখর দাভে।
eng_Latn: I am studying neural machine Translation
ben_Beng: আমি নিউরাল মেশিন ট্রান্সলেশন নিয়ে পড়াশোনা করছি।
eng_Latn: I hail from jodhpur , rajasthan
ben_Beng: আমি রাজস্থানের যোধপুর থেকে এসেছি।
eng_Latn: I am a student at IIT Jodhpur
ben_Beng: আমি আই. আই. টি যোধপুরের ছাত্র।


In [15]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)


en_sents = [
    "Hello , My name is Shikhar Dave",
    "I am studying neural machine Translation",
    "I hail from jodhpur , rajasthan",
    "I am a student at IIT Jodhpur",

]

src_lang, tgt_lang = "eng_Latn", "san_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model


eng_Latn - san_Deva
eng_Latn: Hello , My name is Shikhar Dave
san_Deva: नमस्कारं, मम नाम शिखर् डेव् अस्ति।
eng_Latn: I am studying neural machine Translation
san_Deva: अहं न्यूरल्-मशीन्-अनुवादं अध्ययनं करोति।
eng_Latn: I hail from jodhpur , rajasthan
san_Deva: अहं जोधपुर्, राजस्थानतः आगतः अस्मि।
eng_Latn: I am a student at IIT Jodhpur
san_Deva: अहं जोधपुर-ऐ. ऐ. टि. मध्ये छात्रः अस्मि।


In [13]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)


en_sents = [
    "Hello , My name is Shikhar Dave",
    "I am studying neural machine Translation",
    "I hail from jodhpur , rajasthan",
    "I am a student at IIT Jodhpur",

]

src_lang, tgt_lang = "eng_Latn", "urd_Arab"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



eng_Latn - urd_Arab
eng_Latn: Hello , My name is Shikhar Dave
urd_Arab: ہیلو، میرا نام شیکھر دیو ہے
eng_Latn: I am studying neural machine Translation
urd_Arab: میں نیورل مشین ٹرانسلیشن کا مطالعہ کر رہا ہوں
eng_Latn: I hail from jodhpur , rajasthan
urd_Arab: میں جودھ پور، راجستھان سے ہوں
eng_Latn: I am a student at IIT Jodhpur
urd_Arab: میں آئی آئی ٹی جودھ پور کا طالب علم ہوں۔


In [17]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)

hi_sents = [
    "हैलो, मेरा नाम शिखर डेव है।",
    "मैं तंत्रिका यंत्र अनुवाद का अध्ययन कर रहा हूँ।",
    "मैं जोधपुर, राजस्थान का रहने वाला हूँ।",
]
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model


hin_Deva - eng_Latn
hin_Deva: हैलो, मेरा नाम शिखर डेव है।
eng_Latn: Hello, my name is Shikhar Dave.
hin_Deva: मैं तंत्रिका यंत्र अनुवाद का अध्ययन कर रहा हूँ।
eng_Latn: I am studying Neural Machine Translation.
hin_Deva: मैं जोधपुर, राजस्थान का रहने वाला हूँ।
eng_Latn: I am from Jodhpur, Rajasthan.


In [16]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

hi_sents = [
    "हैलो, मेरा नाम शिखर डेव है।",
    "मैं तंत्रिका यंत्र अनुवाद का अध्ययन कर रहा हूँ।",
    "मैं जोधपुर, राजस्थान का रहने वाला हूँ।",

]
src_lang, tgt_lang = "hin_Deva", "mar_Deva"
mr_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


hin_Deva - mar_Deva
hin_Deva: हैलो, मेरा नाम शिखर डेव है।
mar_Deva: नमस्कार, माझे नाव शिखर डेव्ह आहे.
hin_Deva: मैं तंत्रिका यंत्र अनुवाद का अध्ययन कर रहा हूँ।
mar_Deva: मी न्यूरल मेकॅनिकल ट्रान्सलेशनचा अभ्यास करत आहे.
hin_Deva: मैं जोधपुर, राजस्थान का रहने वाला हूँ।
mar_Deva: मी राजस्थानमधील जोधपूरचा रहिवासी आहे.
